## Iris homebrew network ##

This notebook uses the renown **iris** data set to solve a 3 class classification problem, using a Multi Layer Perceptron with 1 hidden layer and an output layer of 3 *one-hotted* linear units feeding into a *softmax* function. 

The Loss function is *Multi-class Cross Entropy*

You can use the cell below to select either *ReLU* or *sigmoid* functions, and otherwise change the network parameters

*What differences in performance do you notice between ReLU and sigmoid?*

*Why do you think this is?*

You can use this notebook to try out other hidden layer activation functions such as *tanh* or the variations of *ReLU*. You could also add additional hidden layers

Run the notebook cells in the sequence they are given

In [ ]:
#
# Get data from github
#
!wget -nv https://github.com/kcl-bhi-is-01/datasets/raw/main/iris.csv
#

In [ ]:
#
# Main imports and network parameters
#
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import copy
#
iris_df = pd.read_csv("iris.csv")
#
# Parms
#
#hidden_layer_activation = "ReLU"
hidden_layer_activation = "sigmoid"
#
hidden_layer_size = 7
learning_rate     = 0.4
epochs            = 100
#

In [ ]:
#
# Functions
#
def sigmoid(np_matrix):
    #
    # Sigmoid activation
    #
    return 1 / (1 + np.exp(- np_matrix))
#
#
def sigmoid_derivative(np_matrix):
    #
    # Sigmoid derivative
    #
    sig_matrix = sigmoid(np_matrix)
    #
    return np.multiply(sig_matrix, (1 - sig_matrix))
#
#
def ReLU(np_matrix):
    #
    # ReLU activation
    #
    return np.maximum(np_matrix, 0)
#
#
def ReLU_derivative(np_matrix):
    #
    # ReLU derivative
    #
    return np.array(np_matrix == 1).astype(float)
#
#
def softmax(np_vector):
    #
    # softmax
    #
    exp_vect = np.exp(np_vector)
    denom    = np.sum(exp_vect)
    #
    return exp_vect / denom
#
#

In [ ]:
#
# Split
#
from sklearn.model_selection import train_test_split
#
Y = list(iris_df["class"])
X = iris_df.drop("class", axis=1)
#
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
#

In [ ]:
#
# Fit and apply standard scaling
#
from sklearn.preprocessing import StandardScaler
#
scaler = StandardScaler() 
#
scaler.fit(X_train)
#
X_train = pd.DataFrame(scaler.transform(X_train))
X_test  = pd.DataFrame(scaler.transform(X_test))
#
# Reapply column names
#
X_train.columns = X.columns
X_test.columns  = X.columns

In [ ]:
#
# One hot encoding
#
from sklearn.preprocessing import OneHotEncoder
#
enc = OneHotEncoder()
#
Y_train_ar = np.array(Y_train).reshape(-1, 1)
Y_test_ar  = np.array(Y_test).reshape(-1, 1)
#
enc.fit(Y_train_ar)
#
Y_train_oh = enc.transform(Y_train_ar).toarray()
Y_test_oh  = enc.transform(Y_test_ar).toarray()
#

In [ ]:
#
# Set up weights input to hidden
#
w_i_h1 = np.random.uniform(low=-0.3, high=0.3, size=(X_train.shape[1] + 1, hidden_layer_size))
print(w_i_h1.shape)
#
# Set up weights hidden to output
#
w_h1_o = np.random.uniform(low=-0.3, high=0.3, size=(hidden_layer_size + 1, Y_train_oh.shape[1]))
print(w_h1_o.shape)
#

In [ ]:
#
# Set up training data with bias
#
X_train_np = np.concatenate((np.ones([X_train.shape[0], 1]), X_train.to_numpy()), axis=1)
#
train_obs = X_train_np.shape[0]
print(X_train_np.shape)
#
# Get the mean of data in input layer oh no Mrs ...
#
#i_mean = np.mean(X_train_np, axis=0)
#

In [ ]:
#
# Train
#
losses = [0] * epochs
#
for epoch in range(epochs):
    #
    # Forward pass
    #
    h1_sum = np.matmul(X_train_np, w_i_h1)
    #
    # Selection of activation function
    #
    if hidden_layer_activation == "sigmoid":
        h1_act = sigmoid(h1_sum)
    elif hidden_layer_activation == "ReLU":
        h1_act = ReLU(h1_sum)
    #
    # Add bias column to h1 activation matrix (i.e. bias has an activation 1)
    #
    h1_act = np.concatenate((np.ones([X_train_np.shape[0], 1]), h1_act), axis=1)
    #
    o_sum = np.matmul(h1_act, w_h1_o)
    #
    soft_act = np.apply_along_axis(softmax, 1, o_sum)
    #
    # Mean CE loss a bit more meaningful than the sum over all observations
    #
    loss = -np.sum(np.sum(np.multiply(Y_train_oh, np.log(soft_act)), axis=1)) / Y_train_oh.shape[0]
    #
    print("Epoch:", epoch + 1, " Loss: ", round(loss, 6))
    #
    losses[epoch] = loss
    #
    # Backward pass
    #
    # Notes: a) dl/do = softmax vector minus one-hot vector
    #        b) gradients taken as mean across all observations
    #        c) Update weights on each pass
    #
    #
    dl_do = soft_act - Y_train_oh
    #
    # dl/d_w_h1_o = dl/do * do/d_w_h1_o (do/d_w_h1_o = w_h1_o since derivative of weighted sum wrt weights) 
    #
    dl_by_d_w_h1_o = np.zeros([train_obs, w_h1_o.shape[0], w_h1_o.shape[1]])
    #
    dl_by_d_w_i_h1 = np.zeros([train_obs, w_i_h1.shape[0], w_i_h1.shape[1]])
    #
    dl_dh = np.zeros([train_obs, w_h1_o.shape[0]]) 
    #
    # Calculate gradients and weight deltas - hidden to output
    #
    for i in range(train_obs):  # Each observation
        for j in range(w_h1_o.shape[0]): # Each unit in hidden layer (including bias)
            for k in range(w_h1_o.shape[1]): # Each unit in output layer
                #
                dl_dh[i, j] += w_h1_o[j, k] * dl_do[i, k] # Hidden layer gradient
                #
                dl_by_d_w_h1_o[i, j, k] = dl_do[i, k] * h1_act[i, j] # Weight deltas (prior to lr)
            #
        #
    #
    # Calculate weight delta - input to hidden 
    #
    for i in range(train_obs):  # Each observation
        for j in range(w_i_h1.shape[0]): # Each input feature (including bias)
            for k in range(w_i_h1.shape[1]): # Each unit in hidden layer
                #
                dl_by_d_w_i_h1[i, j, k] = dl_dh[i, k] * X_train_np[i, j] # Weight deltas (prior to lr)
            #
        #
    #
    # Update weights
    #
    w_h1_o -= np.mean(dl_by_d_w_h1_o, axis=0) * learning_rate
    w_i_h1 -= np.mean(dl_by_d_w_i_h1, axis=0 )* learning_rate
#

In [ ]:
#
# Plot loss functions
#
plt.plot(losses, color="red")
plt.xlabel("Epoch")
plt.ylabel("Loss")
#
plt.show()
#

In [ ]:
#
# Test - Y_test_oh
#
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
#
X_test_np = np.concatenate((np.ones([X_test.shape[0], 1]), X_test.to_numpy()), axis=1)
#
test_obs = X_test_np.shape[0]
#
h1_sum = np.matmul(X_test_np, w_i_h1)
#
h1_act = np.zeros([test_obs, hidden_layer_size])
#
# Selection of activation function
#
if hidden_layer_activation == "sigmoid":
    h1_act = sigmoid(h1_sum)
elif hidden_layer_activation == "ReLU":
    h1_act = ReLU(h1_sum)
#
# Add bias column to h1 activation matrix (i.e. bias has an activation 1)
#
h1_act = np.concatenate((np.ones([X_test_np.shape[0], 1]), h1_act), axis=1)
#
o_sum = np.matmul(h1_act, w_h1_o)
#
soft_act = np.apply_along_axis(softmax, 1, o_sum)
#
# I use mean CE loss to be a bit more meaningful
#
loss = -np.sum(np.sum(np.multiply(Y_test_oh, np.log(soft_act)), axis=1)) / Y_test_oh.shape[0]
#
# Metrics on test
#
print("Test set Loss: ", round(loss, 6))
#
actual    = np.argmax(Y_test_oh, axis=1)
predicted = np.argmax(soft_act, axis=1)
#
confusion = confusion_matrix(actual, predicted)
conf_disp = ConfusionMatrixDisplay(confusion_matrix=confusion)
conf_disp.plot()
plt.show()
#
print(classification_report(actual, predicted))
# 